In [5]:
# Dokument for at udregne effekten af Danmarks tests
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")

import os

from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [8]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


# for subdir, dirs, files in os.walk(rootdir):
#     if not len(files) == 0:
#         latestdir = subdir
#         latestDate = pd.to_datetime(subdir[-10:])

# print(latestdir)
# print(latestDate)

latestdir = rootdir+ '/SSI_data_2021-05-21'
latestDate = pd.to_datetime(latestdir[-10:])

print(latestdir)
print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-05-21
2021-05-21 00:00:00


# Load datafiler

In [12]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldfAG = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)
dfAG2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
dfAG = fulldfAG.iloc[:-2,:].copy()

# Format columns correctly
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
dfAG["PosPct"] = pd.to_numeric(dfAG["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG["Date"] = dfAG["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    dfAG[name] = pd.to_numeric(dfAG[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))

dfAG2["Dato"] = dfAG2["Dato"].astype('datetime64[D]')
dfAG2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

# Cut off early data in PCR dataframe
dfPCR = dfPCR[dfPCR.Date >= dfAG.iloc[0].Date]

# Also cutoff everything before first of January
firstDate = np.datetime64('2021-02-01')
dfPCR = dfPCR[dfPCR.Date >= firstDate]
dfAG = dfAG[dfAG.Date >= firstDate]

# Cut off last date (since it is not fully done)
dfPCR = dfPCR.iloc[:-1]
dfAG = dfAG.iloc[:-1]

# Get dates
curDates = dfPCR.Date.values.astype('datetime64[D]')
confDates = dfAG2.Dato.values.astype('datetime64[D]')

# Plots

In [13]:
# Overblik over antal tests og antal positive

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1)

PCRpos = dfPCR.NewPositive
AGpos = dfAG.NewPositive
PCRtest = dfPCR.Tested
AGtest = dfAG.Tested
ax1.plot(curDates,PCRtest,label='PCR')
ax1.plot(curDates,AGtest,label='AG')
ax2.plot(curDates,PCRpos,label='PCR')
ax2.plot(curDates,AGpos,label='AG')
# ax2.plot(curDates,np.multiply(PCRpos,PCRtest),label='PCR')
# ax2.plot(curDates,np.multiply(AGpos,AGtest),label='AG')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))


ax1.legend()
ax2.legend()
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …